### Imports

In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
#from IPython.display import display
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

#from nba_api.stats.endpoints import playercareerstats

import os
import time
from bs4 import BeautifulSoup, Comment
from datetime import datetime
#from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [106]:
SEASONS = list(range(2017, 2020))
SEASONS = list(map(str, SEASONS))
#2020 season is an anormaly due to the bubble, so we will scrape it manually, 2021 season started in dec and ended in july
MONTHS = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']

### First attempt at using webdriver for webscraping

In [ ]:
box_score_links = driver.find_elements(By.CSS_SELECTOR, 'td[data-stat="box_score_text"] a')

# Extract the URLs from the links
links = [link.get_attribute("href") for link in box_score_links]


for link in links:
    driver.get(link)
    title = driver.find_element(By.CSS_SELECTOR, 'h1').text
    print(title)

In [ ]:
driver.quit()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Khant Zaw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Khant Zaw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Khant Zaw\AppData\Local\Temp\ipykernel_10048\2924239048.py", line 1, in <cell line: 1>
    driver.quit()
  File "C:\Users\Khant Zaw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python

In [ ]:
links

['https://www.basketball-reference.com/boxscores/202110190MIL.html',
 'https://www.basketball-reference.com/boxscores/202110190LAL.html',
 'https://www.basketball-reference.com/boxscores/202110200CHO.html',
 'https://www.basketball-reference.com/boxscores/202110200DET.html',
 'https://www.basketball-reference.com/boxscores/202110200NYK.html',
 'https://www.basketball-reference.com/boxscores/202110200TOR.html',
 'https://www.basketball-reference.com/boxscores/202110200MEM.html',
 'https://www.basketball-reference.com/boxscores/202110200MIN.html',
 'https://www.basketball-reference.com/boxscores/202110200NOP.html',
 'https://www.basketball-reference.com/boxscores/202110200SAS.html',
 'https://www.basketball-reference.com/boxscores/202110200UTA.html',
 'https://www.basketball-reference.com/boxscores/202110200POR.html',
 'https://www.basketball-reference.com/boxscores/202110200PHO.html',
 'https://www.basketball-reference.com/boxscores/202110210ATL.html',
 'https://www.basketball-reference

In [ ]:
data=[]

for link in links:
    driver.get(link)
    
    # Find the table with id='line_score'
    table_element = driver.find_element(By.CSS_SELECTOR, '#line_score')
    
    # Find the team values
    team_values = table_element.find_elements(By.CSS_SELECTOR, 'th[data-stat="team"]')
    
    # Extract the team abbreviations
    team_abbreviations = [team_value.text for team_value in team_values]

    team1 = team_abbreviations[1]
    team2 = team_abbreviations[2]

    id_list = []
    for team in team_abbreviations[1:3]:
        id_list.append("#box-"+team1+"-game-basic")
        id_list.append("#box-"+team1+"-game-advanced")
    
    for id in id_list:
        table_element = driver.find_element(By.CSS_SELECTOR, id)
        tfoot_element = table_element.find_element(By.CSS_SELECTOR, 'tfoot')
        td_values = tfoot_element.find_elements(By.CSS_SELECTOR, 'td')
        td_texts = [td_value.text for td_value in td_values]
        data.append(td_texts)


In [ ]:
oct_data = []
dummy = ['https://www.basketball-reference.com/boxscores/202110190MIL.html']
for link in dummy:
    response = requests.get(link)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table with id="box-BRK-game-basic"
    table_element = soup.find('table', id="box-BRK-game-basic")

    # Find the tfoot section within the table
    tfoot_element = table_element.find('tfoot')

    # Find all the td values within the tfoot section
    td_values = tfoot_element.find_all('td')

    # Extract the text values from the td elements
    td_texts = [td_value.get_text(strip=True) for td_value in td_values]

    oct_data.append(td_texts)
    

However, I found webdriver to be slow, as such I decided to switch over to BeautifulSoup

### Using BeautifulSoup for web scraping

##### Saving the individual box score html files

In [108]:
for season in SEASONS:
    for month in MONTHS:
        url = "https://www.basketball-reference.com/leagues/NBA_" + season + "_games-" + month + ".html"
        response = requests.get(url)
        print(url)

        # Create a BeautifulSoup object
        soup = BeautifulSoup(response.text, 'html.parser')

        table = soup.find('table', class_='stats_table')

        rows = table.find_all('tr')

        for row in rows:
            # Find the link within each row
            tabdata = row.find('td', attrs={'data-stat': 'box_score_text'})
            if tabdata:
                link = tabdata.find('a')
                boxscore_link = link['href']
                links.append(boxscore_link)
        
        time.sleep(15)

https://www.basketball-reference.com/leagues/NBA_2017_games-october.html


NameError: name 'links' is not defined

In [ ]:
folder_path = r"C:\Users\Khant Zaw\Documents\nba_scores\data\scores"
os.makedirs(folder_path, exist_ok=True)

In [ ]:
ind = 0
for link in links:
    url = "https://www.basketball-reference.com" + link
    print(ind)
    ind += 1
    response = requests.get(url)
    content = response.text
    filename = link.split('/')[-1]
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'w', encoding="utf-8") as file:
        file.write(content)
    time.sleep(6)

1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494


In [ ]:
len(links)

10197

##### Getting the headers and the team abbreviations for further scraping

In [ ]:
for filename in os.listdir(folder_path):
    if filename.endswith('.html'):  # Check if the file is an HTML file
        file_path = os.path.join(folder_path, filename)
        
        # Open the file and read its content
        with open(file_path, 'r', encoding="utf-8") as file:
            html_content = file.read()
        
        soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
div1 = soup.find('div', class_='content_grid')
div2 = div1.find('div')
div3 = div2.find('div', class_="table_wrapper")
comment = div3.find_all(string=lambda text: isinstance(text, Comment))[0]
tabl = BeautifulSoup(comment, 'html.parser')
rows = tabl.select('#line_score tbody tr')
team1_abbr = rows[0].select_one('th a').text
team2_abbr = rows[1].select_one('th a').text
team_abbr = [team1_abbr, team2_abbr]

In [ ]:
div4 = soup.select_one('#div_box-IND-game-basic.table_container')
table = div4.find('table', id='box-IND-game-basic')
headers = [th.text for th in table.select('thead th')]
basic_headers = headers[3:]

In [ ]:
div4 = soup.select_one('#div_box-IND-game-advanced.table_container')
table = div4.find('table', id='box-IND-game-advanced')
headers = [th.text for th in table.select('thead th')]
advanced_headers = headers[3:]

In [ ]:
full_headers = basic_headers+advanced_headers

In [ ]:
df = pd.read_csv("scores_unedited.csv")

In [ ]:
old_headers = list(df.columns)

In [ ]:
old_headers.append("home")

In [ ]:
new_headers = [h + "_opp" for h in old_headers]

In [ ]:
headers = old_headers + new_headers

In [ ]:
headers

['MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 '+/-',
 'MP.1',
 'TS%',
 'eFG%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'ORtg',
 'DRtg',
 'BPM',
 'team',
 'home',
 'MP_opp',
 'FG_opp',
 'FGA_opp',
 'FG%_opp',
 '3P_opp',
 '3PA_opp',
 '3P%_opp',
 'FT_opp',
 'FTA_opp',
 'FT%_opp',
 'ORB_opp',
 'DRB_opp',
 'TRB_opp',
 'AST_opp',
 'STL_opp',
 'BLK_opp',
 'TOV_opp',
 'PF_opp',
 'PTS_opp',
 '+/-_opp',
 'MP.1_opp',
 'TS%_opp',
 'eFG%_opp',
 '3PAr_opp',
 'FTr_opp',
 'ORB%_opp',
 'DRB%_opp',
 'TRB%_opp',
 'AST%_opp',
 'STL%_opp',
 'BLK%_opp',
 'TOV%_opp',
 'USG%_opp',
 'ORtg_opp',
 'DRtg_opp',
 'BPM_opp',
 'team_opp',
 'home_opp']

In [ ]:
full_headers

['MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 '+/-',
 'MP',
 'TS%',
 'eFG%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'ORtg',
 'DRtg',
 'BPM']

In [ ]:
f = os.listdir(folder_path)[0]
file_path = os.path.join(folder_path, f)
with open(file_path, 'r', encoding="utf-8") as file:
    html_content = file.read()
soup = BeautifulSoup(html_content, 'html.parser')
str(int(soup.find_all('u')[1].text[0:4]) + 1)

'2015'

In [ ]:
headers.insert(-1, "date")

In [ ]:
links

['/boxscores/201410280NOP.html',
 '/boxscores/201410280SAS.html',
 '/boxscores/201410280LAL.html',
 '/boxscores/201410290CHO.html',
 '/boxscores/201410290IND.html',
 '/boxscores/201410290TOR.html',
 '/boxscores/201410290MIA.html',
 '/boxscores/201410290BOS.html',
 '/boxscores/201410290NYK.html',
 '/boxscores/201410290MEM.html',
 '/boxscores/201410290DEN.html',
 '/boxscores/201410290UTA.html',
 '/boxscores/201410290PHO.html',
 '/boxscores/201410290SAC.html',
 '/boxscores/201410290POR.html',
 '/boxscores/201410300ORL.html',
 '/boxscores/201410300CLE.html',
 '/boxscores/201410300MIN.html',
 '/boxscores/201410300DAL.html',
 '/boxscores/201410300LAC.html',
 '/boxscores/201410310IND.html',
 '/boxscores/201410310CHI.html',
 '/boxscores/201410310MIL.html',
 '/boxscores/201410310PHO.html',
 '/boxscores/201410310SAC.html',
 '/boxscores/201410310LAL.html',
 '/boxscores/201411010CHO.html',
 '/boxscores/201411010NOP.html',
 '/boxscores/201411010ORL.html',
 '/boxscores/201411010PHI.html',
 '/boxscor

##### Scraping the boxscores

In [ ]:
data_rows2 = []

In [ ]:
for filename in os.listdir(folder_path):
    if filename.endswith('.html'):  # Check if the file is an HTML file
        file_path = os.path.join(folder_path, filename)
        
        # Open the file and read its content
        with open(file_path, 'r', encoding="utf-8") as file:
            html_content = file.read()

        print(filename)
        date = filename[0:8]
        formatted_date = datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")
        print(formatted_date)
        soup = BeautifulSoup(html_content, 'html.parser')
        season = str(int(soup.find_all('u')[1].text[0:4]) + 1)
        div1 = soup.find('div', class_='content_grid')
        div2 = div1.find('div')
        div3 = div2.find('div', class_="table_wrapper")
        comment = div3.find_all(string=lambda text: isinstance(text, Comment))[0]
        tabl = BeautifulSoup(comment, 'html.parser')
        rows = tabl.select('#line_score tbody tr')
        team1_abbr = rows[0].select_one('th a').text
        team2_abbr = rows[1].select_one('th a').text
        team_abbr = [team1_abbr, team2_abbr]
        data_away = []
        data_home = []
        for team in team_abbr:
            basic_div_id = '#div_box-' + team + '-game-basic.table_container'
            div4 = soup.select_one(basic_div_id)
            basic_id = 'box-' + team + '-game-basic'
            table_bas = div4.find('table', id=basic_id)
            for row in table_bas.select('tfoot tr'):
                data_basic = [td.text for td in row.select('td')]
            adv_div_id = '#div_box-' + team + '-game-advanced.table_container'
            div4 = soup.select_one(adv_div_id)
            adv_id = 'box-' + team + '-game-advanced'
            table_adv = div4.find('table', id=adv_id)
            for row in table_adv.select('tfoot tr'):
                data_advanced = [td.text for td in row.select('td')]
            data = data_basic + data_advanced
            data.append(team)
            if (team == team1_abbr): 
                data.append(0)
                data_away = data
            else:
                data.append(1)
                data_home = data
        full_data = data_away + data_home
        full_data2 = data_home + data_away
        full_data.append(season)
        full_data2.append(season)
        full_data.append(formatted_date)
        full_data2.append(formatted_date)
        if (full_data[18] > full_data2[18]):
            full_data.append(1)
            full_data2.append(0)
        else:
            full_data.append(0)
            full_data2.append(1)  
        data_rows2.append(full_data)
        data_rows2.append(full_data2)

201410280LAL.html
2014-10-28
201410280NOP.html
2014-10-28
201410280SAS.html
2014-10-28
201410290BOS.html
2014-10-29
201410290CHO.html
2014-10-29
201410290DEN.html
2014-10-29
201410290IND.html
2014-10-29
201410290MEM.html
2014-10-29
201410290MIA.html
2014-10-29
201410290NYK.html
2014-10-29
201410290PHO.html
2014-10-29
201410290POR.html
2014-10-29
201410290SAC.html
2014-10-29
201410290TOR.html
2014-10-29
201410290UTA.html
2014-10-29
201410300CLE.html
2014-10-30
201410300DAL.html
2014-10-30
201410300LAC.html
2014-10-30
201410300MIN.html
2014-10-30
201410300ORL.html
2014-10-30
201410310CHI.html
2014-10-31
201410310IND.html
2014-10-31
201410310LAL.html
2014-10-31
201410310MIL.html
2014-10-31
201410310PHO.html
2014-10-31
201410310SAC.html
2014-10-31
201411010ATL.html
2014-11-01
201411010CHO.html
2014-11-01
201411010DET.html
2014-11-01
201411010GSW.html
2014-11-01
201411010HOU.html
2014-11-01
201411010MIN.html
2014-11-01
201411010NOP.html
2014-11-01
201411010OKC.html
2014-11-01
201411010ORL.h

In [ ]:
data_rows

[['240',
  '31',
  '73',
  '.425',
  '12',
  '29',
  '.414',
  '34',
  '50',
  '.680',
  '14',
  '33',
  '47',
  '22',
  '7',
  '3',
  '13',
  '30',
  '108',
  '',
  '240',
  '.568',
  '.507',
  '.397',
  '.685',
  '35.9',
  '75.0',
  '56.6',
  '71.0',
  '7.7',
  '4.3',
  '12.0',
  '100.0',
  '118.8',
  '99.0',
  '',
  'HOU',
  0,
  '240',
  '28',
  '79',
  '.354',
  '3',
  '10',
  '.300',
  '31',
  '39',
  '.795',
  '11',
  '25',
  '36',
  '16',
  '7',
  '3',
  '11',
  '32',
  '90',
  '',
  '240',
  '.468',
  '.373',
  '.127',
  '.494',
  '25.0',
  '64.1',
  '43.4',
  '57.1',
  '7.7',
  '6.8',
  '10.3',
  '100.0',
  '99.0',
  '118.8',
  '',
  'LAL',
  1,
  '2014',
  '2014-10-28',
  0],
 ['240',
  '28',
  '79',
  '.354',
  '3',
  '10',
  '.300',
  '31',
  '39',
  '.795',
  '11',
  '25',
  '36',
  '16',
  '7',
  '3',
  '11',
  '32',
  '90',
  '',
  '240',
  '.468',
  '.373',
  '.127',
  '.494',
  '25.0',
  '64.1',
  '43.4',
  '57.1',
  '7.7',
  '6.8',
  '10.3',
  '100.0',
  '99.0',
  '1

In [ ]:
len(data_rows)

20394

### Cleaning the data

In [ ]:
df = pd.DataFrame(data_rows2, columns=headers)

In [ ]:
df.to_csv('scores.csv', index=False)

In [125]:
df = pd.read_csv("scores.csv")

In [126]:
df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,TOV%_opp,USG%_opp,ORtg_opp,DRtg_opp,BPM_opp,team_opp,home_opp,season,date,won
0,240,31,73,0.425,12,29,0.414,34,50,0.680,...,10.3,100,99.0,118.8,NaN,LAL,1,2015,2014/10/28,0
1,240,28,79,0.354,3,10,0.300,31,39,0.795,...,12.0,100,118.8,99.0,NaN,HOU,0,2015,2014/10/28,1
2,240,32,84,0.381,4,11,0.364,16,21,0.762,...,7.3,100,106.1,88.2,NaN,NOP,1,2015,2014/10/28,1
3,240,41,101,0.406,4,17,0.235,15,31,0.484,...,16.2,100,88.2,106.1,NaN,ORL,0,2015,2014/10/28,0
4,240,38,78,0.487,8,21,0.381,16,19,0.842,...,20.6,100,117.3,116.1,NaN,SAS,1,2015,2014/10/28,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20361,240,34,85,0.400,15,38,0.395,14,19,0.737,...,14.1,100,112.6,102.1,NaN,GSW,0,2022,2022/06/10,1
20362,240,31,75,0.413,11,32,0.344,21,31,0.677,...,6.0,100,108.4,97.9,NaN,GSW,1,2022,2022/06/13,1
20363,240,41,88,0.466,9,40,0.225,13,15,0.867,...,16.9,100,97.9,108.4,NaN,BOS,0,2022,2022/06/13,0
20364,240,38,92,0.413,19,46,0.413,8,8,1.000,...,20.5,100,97.7,111.8,NaN,BOS,1,2022,2022/06/16,0


In [127]:
df = df.sort_values("date")
df = df.reset_index(drop=True)

In [128]:
del df["+/-"]
del df["MP.1"]
del df["BPM"]
del df["+/-_opp"]
del df["MP.1_opp"]
del df["BPM_opp"]

In [10]:
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group

df = df.groupby("team", group_keys=False).apply(add_target)

In [130]:
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

C:\Users\Khant Zaw\AppData\Local\Temp\ipykernel_4292\2793819745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


In [131]:
df["won"].value_counts()

0    10183
1    10183
Name: won, dtype: int64

In [132]:
df["target"].value_counts()

0    10168
1    10168
2       30
Name: target, dtype: int64

In [133]:
nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]

In [134]:
nulls

Series([], dtype: int64)

In [135]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [136]:
valid_columns

Index(['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%', 'eFG%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'ORtg', 'DRtg', 'team', 'home', 'MP_opp', 'FG_opp', 'FGA_opp',
       'FG%_opp', '3P_opp', '3PA_opp', '3P%_opp', 'FT_opp', 'FTA_opp',
       'FT%_opp', 'ORB_opp', 'DRB_opp', 'TRB_opp', 'AST_opp', 'STL_opp',
       'BLK_opp', 'TOV_opp', 'PF_opp', 'PTS_opp', 'TS%_opp', 'eFG%_opp',
       '3PAr_opp', 'FTr_opp', 'ORB%_opp', 'DRB%_opp', 'TRB%_opp', 'AST%_opp',
       'STL%_opp', 'BLK%_opp', 'TOV%_opp', 'USG%_opp', 'ORtg_opp', 'DRtg_opp',
       'team_opp', 'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object')

In [137]:
df = df[valid_columns].copy()

In [138]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [139]:
selected_columns

Index(['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%', 'eFG%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'ORtg', 'DRtg', 'home', 'MP_opp', 'FG_opp', 'FGA_opp',
       'FG%_opp', '3P_opp', '3PA_opp', '3P%_opp', 'FT_opp', 'FTA_opp',
       'FT%_opp', 'ORB_opp', 'DRB_opp', 'TRB_opp', 'AST_opp', 'STL_opp',
       'BLK_opp', 'TOV_opp', 'PF_opp', 'PTS_opp', 'TS%_opp', 'eFG%_opp',
       '3PAr_opp', 'FTr_opp', 'ORB%_opp', 'DRB%_opp', 'TRB%_opp', 'AST%_opp',
       'STL%_opp', 'BLK%_opp', 'TOV%_opp', 'USG%_opp', 'ORtg_opp', 'DRtg_opp',
       'home_opp'],
      dtype='object')

In [ ]:
scaler = MinMaxScaler()

In [140]:

df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [141]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

def find_team_averages(team):
    rolling = team.rolling(5).mean()
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

C:\Users\Khant Zaw\AppData\Local\Temp\ipykernel_4292\114307051.py:4: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['team'], dtype='object')
  rolling = team.rolling(5).mean()


In [142]:
df_rolling

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,AST%_opp,STL%_opp,BLK%_opp,TOV%_opp,USG%_opp,ORtg_opp,DRtg_opp,home_opp,won,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20361,0.0,0.409091,0.359420,0.397608,0.517241,0.493939,0.481473,0.330233,0.301587,0.723454,...,0.506048,0.423963,0.247273,0.422388,0.0,0.445225,0.454885,0.6,0.8,2022.0
20362,0.0,0.390909,0.330435,0.398086,0.517241,0.493939,0.481473,0.339535,0.323810,0.696616,...,0.542335,0.472811,0.222857,0.411194,0.0,0.465862,0.443249,0.6,1.0,2022.0
20363,0.0,0.454545,0.379710,0.439713,0.503448,0.560606,0.421853,0.269767,0.238095,0.758926,...,0.688045,0.282949,0.329351,0.485821,0.0,0.443249,0.465862,0.4,0.0,2022.0
20364,0.0,0.450000,0.391304,0.425359,0.503448,0.563636,0.419715,0.255814,0.215873,0.821237,...,0.695640,0.294009,0.347013,0.552985,0.0,0.377168,0.458836,0.6,0.0,2022.0


In [143]:
rolling_cols = [f"{col}_5" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [144]:
df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,AST%_opp_5,STL%_opp_5,BLK%_opp_5,TOV%_opp_5,USG%_opp_5,ORtg_opp_5,DRtg_opp_5,home_opp_5,won_5,season_5
0,0.0,0.272727,0.188406,0.373206,0.413793,0.378788,0.491686,0.767442,0.777778,0.626604,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.204545,0.275362,0.203349,0.103448,0.090909,0.356295,0.697674,0.603175,0.760793,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.295455,0.347826,0.267943,0.137931,0.106061,0.432304,0.348837,0.317460,0.722287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.500000,0.594203,0.327751,0.137931,0.196970,0.279097,0.325581,0.476190,0.397900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.431818,0.260870,0.521531,0.275862,0.257576,0.452494,0.348837,0.285714,0.815636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20361,0.0,0.340909,0.362319,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.506048,0.423963,0.247273,0.422388,0.0,0.445225,0.454885,0.6,0.8,2022.0
20362,0.0,0.272727,0.217391,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.542335,0.472811,0.222857,0.411194,0.0,0.465862,0.443249,0.6,1.0,2022.0
20363,0.0,0.500000,0.405797,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.688045,0.282949,0.329351,0.485821,0.0,0.443249,0.465862,0.4,0.0,2022.0
20364,0.0,0.431818,0.463768,0.344498,0.655172,0.636364,0.490499,0.162791,0.111111,1.000000,...,0.695640,0.294009,0.347013,0.552985,0.0,0.377168,0.458836,0.6,0.0,2022.0


In [145]:
df = df.dropna()

In [146]:
df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,AST%_opp_5,STL%_opp_5,BLK%_opp_5,TOV%_opp_5,USG%_opp_5,ORtg_opp_5,DRtg_opp_5,home_opp_5,won_5,season_5
97,0.0,0.340909,0.072464,0.607656,0.206897,0.257576,0.339667,0.302326,0.380952,0.486581,...,0.540084,0.355760,0.267532,0.358955,0.0,0.475521,0.346213,0.8,0.6,2015.0
104,0.0,0.363636,0.115942,0.588517,0.586207,0.500000,0.545131,0.465116,0.412698,0.740957,...,0.457103,0.443318,0.281039,0.435821,0.0,0.358946,0.531284,0.8,0.0,2015.0
105,0.0,0.363636,0.405797,0.308612,0.206897,0.181818,0.445368,0.674419,0.634921,0.687281,...,0.644444,0.323502,0.138701,0.464925,0.0,0.582437,0.412953,0.4,0.4,2015.0
112,0.0,0.477273,0.289855,0.552632,0.310345,0.196970,0.628266,0.279070,0.253968,0.725788,...,0.454290,0.388940,0.144935,0.611940,0.0,0.278595,0.367508,0.6,0.8,2015.0
113,0.0,0.295455,0.130435,0.466507,0.275862,0.257576,0.452494,0.418605,0.317460,0.889148,...,0.478200,0.376959,0.177143,0.416418,0.0,0.437980,0.422832,0.4,0.8,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20361,0.0,0.340909,0.362319,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.506048,0.423963,0.247273,0.422388,0.0,0.445225,0.454885,0.6,0.8,2022.0
20362,0.0,0.272727,0.217391,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.542335,0.472811,0.222857,0.411194,0.0,0.465862,0.443249,0.6,1.0,2022.0
20363,0.0,0.500000,0.405797,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.688045,0.282949,0.329351,0.485821,0.0,0.443249,0.465862,0.4,0.0,2022.0
20364,0.0,0.431818,0.463768,0.344498,0.655172,0.636364,0.490499,0.162791,0.111111,1.000000,...,0.695640,0.294009,0.347013,0.552985,0.0,0.377168,0.458836,0.6,0.0,2022.0


### Training the model

In [155]:

rr = RidgeClassifier(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=30, 
                                direction="backward",
                                cv=split,
                                n_jobs=1
                               )

In [156]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30, n_jobs=1)

In [149]:
predictors = list(selected_columns[sfs.get_support()])

In [150]:
predictors

['3P%',
 'FT',
 'FTA',
 'ORB',
 'TRB',
 'AST',
 'PF',
 'TS%',
 '3PAr',
 'ORB%',
 'TRB%',
 'AST%',
 'STL%',
 'MP_opp',
 'FGA_opp',
 'FG%_opp',
 '3P_opp',
 '3PA_opp',
 '3P%_opp',
 'FT_opp',
 'FTA_opp',
 'FT%_opp',
 'ORB_opp',
 'DRB_opp',
 'TOV_opp',
 'FTr_opp',
 'ORB%_opp',
 'TRB%_opp',
 'AST%_opp',
 'STL%_opp']

In [9]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [152]:
predictions = backtest(df, rr, predictors)

In [153]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.5142440244580322

In [154]:
df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

home
0.0    0.491194
1.0    0.509023
dtype: float64

At this point, our model marginally performs better than the baseline, so there is still room for improvement

In [167]:
split = TimeSeriesSplit(n_splits=5)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=20, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )

In [168]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                          direction='backward',
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=20, n_jobs=1)

In [169]:
predictors = list(selected_columns[sfs.get_support()])

In [170]:
predictions = backtest(df, rr, predictors)
accuracy_score(predictions["actual"], predictions["prediction"])

0.5168148971650918

In [158]:
gb = GradientBoostingClassifier()

In [172]:
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(gb, 
                                n_features_to_select=30, 
                                direction="backward",
                                cv=split,
                                n_jobs=1
                               )

In [173]:
sfs.fit(df[selected_columns], df["target"])
predictors = list(selected_columns[sfs.get_support()])

In [174]:
predictions_gb = backtest(df, gb, predictors)
accuracy_score(predictions_gb["actual"], predictions_gb["prediction"])

0.5091022790439133

In [175]:
rfc = RandomForestClassifier()

In [176]:
sfs = SequentialFeatureSelector(rfc, 
                                n_features_to_select=20, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )
sfs.fit(df[selected_columns], df["target"])
predictors = list(selected_columns[sfs.get_support()])

predictions_rfc = backtest(df, rfc, predictors)
accuracy_score(predictions_rfc["actual"], predictions_rfc["prediction"])

0.5038215675375208

In [13]:
df = pd.read_csv("scores.csv")

In [14]:
df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,TOV%_opp,USG%_opp,ORtg_opp,DRtg_opp,BPM_opp,team_opp,home_opp,season,date,won
0,240,31,73,0.425,12,29,0.414,34,50,0.680,...,10.3,100,99.0,118.8,NaN,LAL,1,2015,2014/10/28,0
1,240,28,79,0.354,3,10,0.300,31,39,0.795,...,12.0,100,118.8,99.0,NaN,HOU,0,2015,2014/10/28,1
2,240,32,84,0.381,4,11,0.364,16,21,0.762,...,7.3,100,106.1,88.2,NaN,NOP,1,2015,2014/10/28,1
3,240,41,101,0.406,4,17,0.235,15,31,0.484,...,16.2,100,88.2,106.1,NaN,ORL,0,2015,2014/10/28,0
4,240,38,78,0.487,8,21,0.381,16,19,0.842,...,20.6,100,117.3,116.1,NaN,SAS,1,2015,2014/10/28,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20361,240,34,85,0.400,15,38,0.395,14,19,0.737,...,14.1,100,112.6,102.1,NaN,GSW,0,2022,2022/06/10,1
20362,240,31,75,0.413,11,32,0.344,21,31,0.677,...,6.0,100,108.4,97.9,NaN,GSW,1,2022,2022/06/13,1
20363,240,41,88,0.466,9,40,0.225,13,15,0.867,...,16.9,100,97.9,108.4,NaN,BOS,0,2022,2022/06/13,0
20364,240,38,92,0.413,19,46,0.413,8,8,1.000,...,20.5,100,97.7,111.8,NaN,BOS,1,2022,2022/06/16,0


In [15]:
df = df.sort_values("date")
df = df.reset_index(drop=True)

In [16]:
del df["+/-"]
del df["MP.1"]
del df["BPM"]
del df["+/-_opp"]
del df["MP.1_opp"]
del df["BPM_opp"]

In [17]:
df = df.groupby("team", group_keys=False).apply(add_target)

In [18]:
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

C:\Users\Khant Zaw\AppData\Local\Temp\ipykernel_7688\2793819745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


In [19]:
df["won"].value_counts()

0    10183
1    10183
Name: won, dtype: int64

In [20]:
df["target"].value_counts()

0    10168
1    10168
2       30
Name: target, dtype: int64

In [21]:
df = df[df["target"] != 2]

In [22]:
df["target"].value_counts()

0    10168
1    10168
Name: target, dtype: int64

In [23]:
nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]

In [24]:
nulls

Series([], dtype: int64)

In [25]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [26]:
valid_columns

Index(['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%', 'eFG%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'ORtg', 'DRtg', 'team', 'home', 'MP_opp', 'FG_opp', 'FGA_opp',
       'FG%_opp', '3P_opp', '3PA_opp', '3P%_opp', 'FT_opp', 'FTA_opp',
       'FT%_opp', 'ORB_opp', 'DRB_opp', 'TRB_opp', 'AST_opp', 'STL_opp',
       'BLK_opp', 'TOV_opp', 'PF_opp', 'PTS_opp', 'TS%_opp', 'eFG%_opp',
       '3PAr_opp', 'FTr_opp', 'ORB%_opp', 'DRB%_opp', 'TRB%_opp', 'AST%_opp',
       'STL%_opp', 'BLK%_opp', 'TOV%_opp', 'USG%_opp', 'ORtg_opp', 'DRtg_opp',
       'team_opp', 'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object')

In [27]:
df = df[valid_columns].copy()

In [28]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [29]:
selected_columns

Index(['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%', 'eFG%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'ORtg', 'DRtg', 'home', 'MP_opp', 'FG_opp', 'FGA_opp',
       'FG%_opp', '3P_opp', '3PA_opp', '3P%_opp', 'FT_opp', 'FTA_opp',
       'FT%_opp', 'ORB_opp', 'DRB_opp', 'TRB_opp', 'AST_opp', 'STL_opp',
       'BLK_opp', 'TOV_opp', 'PF_opp', 'PTS_opp', 'TS%_opp', 'eFG%_opp',
       '3PAr_opp', 'FTr_opp', 'ORB%_opp', 'DRB%_opp', 'TRB%_opp', 'AST%_opp',
       'STL%_opp', 'BLK%_opp', 'TOV%_opp', 'USG%_opp', 'ORtg_opp', 'DRtg_opp',
       'home_opp'],
      dtype='object')

In [31]:
scaler = MinMaxScaler()

In [32]:
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [33]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

def find_team_averages(team):
    rolling = team.rolling(10).mean()
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

C:\Users\Khant Zaw\AppData\Local\Temp\ipykernel_7688\1780157074.py:4: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['team'], dtype='object')
  rolling = team.rolling(10).mean()


In [34]:
df_rolling

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,AST%_opp,STL%_opp,BLK%_opp,TOV%_opp,USG%_opp,ORtg_opp,DRtg_opp,home_opp,won,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20359,0.0,0.386364,0.279710,0.441627,0.455172,0.471212,0.439667,0.458140,0.393651,0.765694,...,0.523207,0.429493,0.249091,0.389179,0.0,0.433370,0.490670,0.6,0.5,2022.0
20360,0.0,0.502273,0.382609,0.500478,0.493103,0.501515,0.458789,0.309302,0.276190,0.741657,...,0.626864,0.307834,0.271948,0.352612,0.0,0.477717,0.544237,0.4,0.3,2022.0
20361,0.0,0.381818,0.288406,0.428230,0.468966,0.477273,0.448100,0.434884,0.373016,0.764177,...,0.528551,0.430415,0.207273,0.400000,0.0,0.423161,0.481449,0.5,0.6,2022.0
20362,0.0,0.354545,0.275362,0.404545,0.437931,0.465152,0.429572,0.434884,0.385714,0.736639,...,0.529677,0.453917,0.201818,0.374627,0.0,0.423052,0.440615,0.5,0.6,2022.0


In [35]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [36]:
df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,AST%_opp_10,STL%_opp_10,BLK%_opp_10,TOV%_opp_10,USG%_opp_10,ORtg_opp_10,DRtg_opp_10,home_opp_10,won_10,season_10
0,0.0,0.272727,0.188406,0.373206,0.413793,0.378788,0.491686,0.767442,0.777778,0.626604,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.204545,0.275362,0.203349,0.103448,0.090909,0.356295,0.697674,0.603175,0.760793,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.295455,0.347826,0.267943,0.137931,0.106061,0.432304,0.348837,0.317460,0.722287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.500000,0.594203,0.327751,0.137931,0.196970,0.279097,0.325581,0.476190,0.397900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.431818,0.260870,0.521531,0.275862,0.257576,0.452494,0.348837,0.285714,0.815636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20359,0.0,0.545455,0.420290,0.511962,0.448276,0.469697,0.440618,0.372093,0.365079,0.659277,...,0.523207,0.429493,0.249091,0.389179,0.0,0.433370,0.490670,0.6,0.5,2022.0
20360,0.0,0.477273,0.449275,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,...,0.626864,0.307834,0.271948,0.352612,0.0,0.477717,0.544237,0.4,0.3,2022.0
20361,0.0,0.340909,0.362319,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.528551,0.430415,0.207273,0.400000,0.0,0.423161,0.481449,0.5,0.6,2022.0
20362,0.0,0.272727,0.217391,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.529677,0.453917,0.201818,0.374627,0.0,0.423052,0.440615,0.5,0.6,2022.0


In [37]:
df = df.dropna()

In [38]:
df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,AST%_opp_10,STL%_opp_10,BLK%_opp_10,TOV%_opp_10,USG%_opp_10,ORtg_opp_10,DRtg_opp_10,home_opp_10,won_10,season_10
244,0.00,0.363636,0.289855,0.404306,0.344828,0.303030,0.495249,0.465116,0.412698,0.740957,...,0.476653,0.385714,0.248831,0.387687,0.0,0.403622,0.369045,0.6,0.7,2015.0
246,0.25,0.431818,0.521739,0.303828,0.413793,0.484848,0.395487,0.000000,0.079365,0.028005,...,0.555977,0.376959,0.220000,0.369030,0.0,0.419319,0.364435,0.6,0.3,2015.0
248,0.00,0.454545,0.318841,0.495215,0.172414,0.060606,0.742280,0.372093,0.301587,0.824971,...,0.581575,0.350691,0.151948,0.462687,0.0,0.483425,0.407135,0.4,0.5,2015.0
250,0.00,0.386364,0.144928,0.586124,0.344828,0.272727,0.540380,0.441860,0.380952,0.766628,...,0.393530,0.401843,0.195844,0.369030,0.0,0.501866,0.449067,0.6,0.6,2015.0
254,0.00,0.340909,0.347826,0.325359,0.275862,0.272727,0.432304,0.232558,0.333333,0.416569,...,0.557947,0.452995,0.214286,0.368284,0.0,0.392865,0.360373,0.4,0.5,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20359,0.00,0.545455,0.420290,0.511962,0.448276,0.469697,0.440618,0.372093,0.365079,0.659277,...,0.523207,0.429493,0.249091,0.389179,0.0,0.433370,0.490670,0.6,0.5,2022.0
20360,0.00,0.477273,0.449275,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,...,0.626864,0.307834,0.271948,0.352612,0.0,0.477717,0.544237,0.4,0.3,2022.0
20361,0.00,0.340909,0.362319,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.528551,0.430415,0.207273,0.400000,0.0,0.423161,0.481449,0.5,0.6,2022.0
20362,0.00,0.272727,0.217391,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.529677,0.453917,0.201818,0.374627,0.0,0.423052,0.440615,0.5,0.6,2022.0


In [40]:
split = TimeSeriesSplit(n_splits=3)

rr = RidgeClassifier(alpha=1)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=30, 
                                direction="backward",
                                cv=split,
                                n_jobs=1
                               )

In [41]:
sfs.fit(df[selected_columns], df["target"])
predictors = list(selected_columns[sfs.get_support()])
predictions = backtest(df, rr, predictors)
accuracy_score(predictions["actual"], predictions["prediction"])

0.5121824394592185

In [48]:
from xgboost import XGBClassifier

In [53]:
xgb = XGBClassifier(tree_method='gpu_hist')

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(xgb, 
                                n_features_to_select=20, 
                                direction="backward",
                                cv=split,
                                n_jobs=1
                               )



In [46]:
sfs

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                          direction='backward',
                          estimator=XGBClassifier(base_score=None, booster=None,
                                                  callbacks=None,
                                                  colsample_bylevel=None,
                                                  colsample_bynode=None,
                                                  colsample_bytree=None,
                                                  early_stopping_rounds=None,
                                                  enable_categorical=False,
                                                  eval_metric=None,
                                                  feature_types=None,
                                                  gamma=No...
                                                  importance_type=None,
                                                  interac

In [54]:
sfs.fit(df[selected_columns], df["target"])
predictors = list(selected_columns[sfs.get_support()])

In [55]:
predictions_xgb = backtest(df, xgb, predictors)
accuracy_score(predictions_xgb["actual"], predictions_xgb["prediction"])

0.5002971326697371

### Conclusion
The most accurate model I could come up with is using ridge classification, with 20 features selected in the forward direction and its accuray score is 0.519 which is marginally better than the home winning rate of 0.509. 
An extension to this project to increase the accuracy could be to include other data, such as the best player's statistics or to use a different model and optimise it.